### In-Game Purchase Data Analysis

#### Background 

In this homework, you are ask to deal with a task of analyzing an "in-game purchase" dataset. In this game, players are able to to purchase optional items that enhance their playing experience. Now your task is to generate a report that breaks down the game's purchasing data into meaningful insights.

-----

#### Observable Trends

* There are 1163 active players. The vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%). 

* The age group that spends the most money is the 20-24 with 1,114.06 dollars as total purchase value and an average purchase of 4.32. In contrast, the demographic group that has the highest average purchase is the 35-39 with 4.76 and a total purchase value of 147.67. 
-----


In [274]:
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [275]:
total_players_count = purchase_data['SN'].nunique()

total_players = pd.DataFrame({'Total Players': [total_players_count]})
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [276]:
items_count = purchase_data['Item ID'].nunique()

unique_items_data = purchase_data.drop_duplicates(subset='Item ID')
average_price = unique_items_data['Price'].mean()

purchase_count = purchase_data['Purchase ID'].nunique()

total_revenue = purchase_data['Price'].sum()

purchase_analysis = pd.DataFrame({'Number of Unique Items': [items_count], 
                                 'Average Price': [average_price], 
                                 'Number of Purchases': [purchase_count],
                                 'Total Revenue': [total_revenue]})

purchase_analysis['Average Price'] = purchase_analysis['Average Price'].apply(lambda x: f'${x:.2f}')
purchase_analysis['Total Revenue'] = purchase_analysis['Total Revenue'].apply(lambda x: f'${x:,.2f}')

purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [277]:
unique_player_data = purchase_data.drop_duplicates(subset='SN')
unique_player_count = unique_player_data.shape[0]
gender_data = unique_player_data['Gender']

male_count = gender_data[purchase_data['Gender'] == 'Male'].shape[0]
male_per = (male_count / unique_player_count) * 100

female_count = gender_data[purchase_data['Gender'] == 'Female'].shape[0]
female_per = (female_count / unique_player_count) * 100

other_count = gender_data[purchase_data['Gender'] == 'Other / Non-Disclosed'].shape[0]
other_per = (other_count / unique_player_count) * 100

gender_summary = pd.DataFrame({
    'Percentage of Players': [round(male_per, 2), round(female_per, 2), round(other_per, 2)],
    'Total Count': [male_count, female_count, other_count]
}, index=['Male', 'Female', 'Other / Non-Disclosed'])

gender_summary 

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain below values by gender
  * Purchase Count
  * Average Purchase Price  
  * Total Purchase Value
  * Average Purchase Total per Person by Gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [278]:
purchase_analysis_by_gender = purchase_data.groupby('Gender').agg({
    'Purchase ID': 'count',  
    'Price': ['mean', 'sum']})

purchase_analysis_by_gender.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']

unique_player_count_by_gender = unique_player_data['Gender'].value_counts()
purchase_analysis_by_gender['Average Purchase Total per Person'] = (
    purchase_analysis_by_gender['Total Purchase Value'] / unique_player_count_by_gender)

purchase_analysis_by_gender['Average Purchase Price'] = purchase_analysis_by_gender['Average Purchase Price'].round(5)
purchase_analysis_by_gender['Total Purchase Value'] = purchase_analysis_by_gender['Total Purchase Value'].round(2)
purchase_analysis_by_gender['Average Purchase Total per Person'] = purchase_analysis_by_gender['Average Purchase Total per Person'].apply(lambda x: f'${x:,.2f}')

purchase_analysis_by_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,
Female,113,3.20301,361.94,$4.47
Male,652,3.01785,1967.64,$4.07
Other / Non-Disclosed,15,3.34600,50.19,$4.56


## Age Demographics

* Categorize players using the age groups. Hint: use pd.cut()
  * each group is for 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* round the percentage column to two decimal points
* Display Age Demographics Table

In [279]:
player_age_data = pd.DataFrame({'Age': unique_player_data['Age']})

cut = [0, 9, 14, 19, 24, 29, 34, 39, 200]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

player_age_data['range'] = pd.cut(player_age_data['Age'], bins=cut, labels=labels)
grouped_data = player_age_data.groupby('range')['Age'].count().reset_index(name='Total Count')

# Calculate percentage of players per range
total_players = player_age_data['Age'].count()
grouped_data['Percentage of Players'] = (grouped_data['Total Count'] / total_players) * 100

# 需要先設index之後才能對應
grouped_data = grouped_data.set_index('range').reindex(labels)

result = pd.DataFrame({
    'Percentage of Players': grouped_data['Percentage of Players'].round(2),
    'Total Count': grouped_data['Total Count']
}, index= ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'])

result 

/var/folders/95/hw148ws149q824gl59r9kf400000gn/T/ipykernel_3969/1236679341.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = player_age_data.groupby('range')['Age'].count().reset_index(name='Total Count')


,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


## Purchasing Analysis (Age)

* The below each broken into groups of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [280]:
cut = [0, 9, 14, 19, 24, 29, 34, 39, 200]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchase_data['Age Group'] = pd.cut(purchase_data['Age'], bins=cut, labels=labels)

purchase_count = purchase_data.groupby('Age Group', observed=True)['Purchase ID'].count()

avg_purchase_price = purchase_data.groupby('Age Group', observed=True)['Price'].mean()

total_purchase_value = purchase_data.groupby('Age Group', observed=True)['Price'].sum()

# Calculate Average Purchase Total per Person by Age Group 
unique_players_by_age = purchase_data.groupby('Age Group', observed=True)['SN'].nunique()
avg_purchase_per_person = total_purchase_value / unique_players_by_age

age_purchase_summary = pd.DataFrame({
    'Purchase Count': purchase_count,
    'Average Purchase Price': avg_purchase_price.round(2).apply(lambda x: f'${x:,.2f}'),
    'Total Purchase Value': total_purchase_value.round(2).apply(lambda x: f'${x:,.2f}'),
    'Average Purchase Total per Person': avg_purchase_per_person.round(2).apply(lambda x: f'${x:,.2f}')
}, index= ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'])

age_purchase_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value (top-5 players with highest total purchase value), then list (in a table):
  * SN(screen name)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [281]:
spender_data = purchase_data.groupby('SN').agg({
    'Purchase ID': 'count',  
    'Price': ['mean', 'sum']})

spender_data.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']

top_spenders = spender_data.sort_values(by='Total Purchase Value', ascending=False).head(5)

top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].round(2).apply(lambda x: f'${x:,.2f}')
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].round(2).apply(lambda x: f'${x:,.2f}')

top_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Top 5 most popular items by purchase count:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [294]:
item_data = purchase_data.groupby(['Item ID', 'Item Name']).agg({
    'Purchase ID': 'count', 
    'Price': ['mean', 'sum'] })

item_data.columns = ['Purchase Count', 'Average Item Price', 'Total Purchase Value']

item_data['Average Item Price'] = item_data['Average Item Price'].round(2).apply(lambda x: f'${x:,.2f}')
item_data['Total Purchase Value'] = item_data['Total Purchase Value'].round(2).apply(lambda x: f'${x:,.2f}')

top_item = item_data.sort_values(by='Purchase Count', ascending=False).head(5)

top_item

,,Purchase Count,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Top 5 most profitable items by total purchase value:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Sort the above table by total purchase value in descending order
* Display a preview of the data frame

In [300]:
item_data = purchase_data.groupby(['Item ID', 'Item Name']).agg({
    'Purchase ID': 'count', 
    'Price': ['mean', 'sum'] })

item_data.columns = ['Purchase Count', 'Item Price', 'Total Purchase Value']

top_item = item_data.sort_values(by='Total Purchase Value', ascending=False).head(5)

top_item['Item Price'] = top_item['Item Price'].round(2).apply(lambda x: f'${x:,.2f}')
top_item['Total Purchase Value'] = top_item['Total Purchase Value'].round(2).apply(lambda x: f'${x:,.2f}')

top_item

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
